In [2]:
#Librerias
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re

In [17]:
def read_file(file):
    fields = ['DIAG_INI', 'EGRESO','ENTIDAD','MUNIC','SEXO','EDAD']
    df = pd.read_csv(file, skipinitialspace=True, usecols=fields)
    df.keys()
    dff = df[df.DIAG_INI.str.contains('^C', regex= True, na=False)]
    return dff


In [18]:
def add_icd(df):
    currDict = pd.read_excel('CODEX.xlsx')
    currDict.set_index('cause', inplace=True)
    cause_dict = currDict.to_dict()['ICD']
    df['ICD'] = df['DIAG_INI'].map(cause_dict)
    MexMuerto = pd.read_excel('types.xlsx')
    MexMuerto.set_index('ICD', inplace=True)
    MexMuerto = MexMuerto.to_dict()['organ']
    df['organ'] = df['ICD'].map(MexMuerto)
    return df

In [19]:
def add_states(dff):
    dff.SEXO = dff.SEXO.astype(int)
    dff.ENTIDAD = dff.ENTIDAD.astype(int)
    dff.MUNIC = dff.MUNIC.astype(int)
    dff.EDAD = dff.EDAD.astype(int)
    dff['ENTIDAD'] = dff['ENTIDAD'].apply(lambda x: '{0:0>2}'.format(x))
    dff['MUNIC'] = dff['MUNIC'].apply(lambda x: '{0:0>3}'.format(x))
    index_names = dff[((dff['ENTIDAD'] == '99') & (dff['MUNIC'] == '999'))].index
    dff.drop(index_names, inplace = True) 
    from os.path import dirname
    stateDict = pd.read_excel('states.xlsx')
    stateDict['code'] = stateDict['code'].apply(lambda x: '{0:0>2}'.format(x))
    stateDict.set_index('code', inplace=True)
    cause_dict = stateDict.to_dict()['state']
    new_col = dff['ENTIDAD'].map(cause_dict)
    dff.insert(loc=4, column='ENT_MX', value=new_col)
    return dff

In [20]:
def add_muns(dff):
    from os.path import dirname
    stateDict = pd.read_excel('states.xlsx')
    stateDict['code'] = stateDict['code'].apply(lambda x: '{0:0>2}'.format(x))
    stateDict.set_index('code', inplace=True)
    cause_dict = stateDict.to_dict()['state']
    claves = list()
    for key in cause_dict.keys():
        claves.append(key)
    currDict = pd.read_excel('Municipalities.xlsx')
    currDict['Clave_Entidad'] = currDict['Clave_Entidad'].apply(lambda x: '{0:0>2}'.format(x))
    currDict['Clave_Mun'] = currDict['Clave_Mun'].apply(lambda x: '{0:0>3}'.format(x))
    df_ = pd.DataFrame(dff['ENTIDAD'])
    del df_['ENTIDAD']
    muns = list()
    cols = list()
    i=0
    for item in claves:
        mun = currDict.loc[(currDict['Clave_Entidad'] == item)]
        mun.set_index('Clave_Mun', inplace=True)
        currMun = mun.to_dict()['Nombre_Municipio']
        new_col = dff.loc[dff['ENTIDAD'] == item]['MUNIC'].map(currMun)
        name = "col"+str(i)
        df_.insert(i, column=name, value=new_col)
        muns.append(currMun)
        cols.append(name)
        i = i+1
    df_.fillna('', inplace=True)
    df_['combined'] = df_[cols].apply(lambda row: ''.join(row.values.astype(str)), axis=1)
    new_col = df_['combined']
    dff.insert(6, column='MUNIC_MX', value=new_col)
    return dff

In [22]:
def add_timestamp(dff):
    index_names = dff[((dff['EGRESO'] == 99) | (dff['EGRESO'] == 999))].index
    dff.drop(index_names, inplace = True) 
    dff['EGRESO'] =  pd.to_datetime(dff['EGRESO'], yearfirst = True, errors = 'raise')
    dff.reset_index(inplace=True)
    dff.drop(columns=['index'],inplace=True)
    return dff

In [23]:
dirs = list()
from os.path import dirname
dir = os.getcwd()+"\\Egresos"
for filename in os.listdir(dir):
    if filename.endswith(".csv") or filename.endswith(".CSV"):
        dirs.append(dir+"\\"+filename)
dirs

['c:\\Users\\David Fernandez\\Desktop\\pruebasInci\\Egresos\\EGRESO_2010.csv',
 'c:\\Users\\David Fernandez\\Desktop\\pruebasInci\\Egresos\\EGRESO_2011.csv',
 'c:\\Users\\David Fernandez\\Desktop\\pruebasInci\\Egresos\\EGRESO_2012.csv',
 'c:\\Users\\David Fernandez\\Desktop\\pruebasInci\\Egresos\\EGRESO_2013.csv',
 'c:\\Users\\David Fernandez\\Desktop\\pruebasInci\\Egresos\\EGRESO_2014.csv',
 'c:\\Users\\David Fernandez\\Desktop\\pruebasInci\\Egresos\\EGRESO_2015.csv',
 'c:\\Users\\David Fernandez\\Desktop\\pruebasInci\\Egresos\\EGRESO_2016.csv',
 'c:\\Users\\David Fernandez\\Desktop\\pruebasInci\\Egresos\\EGRESO_2018.csv',
 'c:\\Users\\David Fernandez\\Desktop\\pruebasInci\\Egresos\\EGRESO_2019.csv']

In [24]:
for filename in dirs:
    print(filename)
    m = re.search('(\d{4})', filename)
    currDF = read_file(filename)
    icd = add_icd(currDF)
    states = add_states(icd)
    muns = add_muns(states)
    time = add_timestamp(muns)
    name = 'Cleaned\Inc_'+m.group(1)+'.csv' 
    time.to_csv(name)

c:\Users\David Fernandez\Desktop\pruebasInci\Egresos\EGRESO_2010.csv
c:\Users\David Fernandez\Desktop\pruebasInci\Egresos\EGRESO_2011.csv
c:\Users\David Fernandez\Desktop\pruebasInci\Egresos\EGRESO_2012.csv
c:\Users\David Fernandez\Desktop\pruebasInci\Egresos\EGRESO_2013.csv
c:\Users\David Fernandez\Desktop\pruebasInci\Egresos\EGRESO_2014.csv
c:\Users\David Fernandez\Desktop\pruebasInci\Egresos\EGRESO_2015.csv
c:\Users\David Fernandez\Desktop\pruebasInci\Egresos\EGRESO_2016.csv
c:\Users\David Fernandez\Desktop\pruebasInci\Egresos\EGRESO_2018.csv
c:\Users\David Fernandez\Desktop\pruebasInci\Egresos\EGRESO_2019.csv


### 2017 

In [25]:
def read_file_17():
    file = os.getcwd()+"\\egresos_new\\EGRESO_2017.csv"
    fields = ['DIAG_INI', 'EGRESO','ENTIDAD','MUNIC','SEXO','EDAD']
    df = pd.read_csv(file, sep='|', skipinitialspace=True, usecols=fields)
    df.keys()
    dff = df[df.DIAG_INI.str.contains('^C', regex= True, na=False)]
    return dff


In [26]:
currDF = read_file_17()
icd = add_icd(currDF)
states = add_states(icd)
muns = add_muns(states)
time = add_timestamp(muns)
name = 'Cleaned\Inc_2017.csv' 
time.to_csv(name)

In [3]:
dirs = list()
from os.path import dirname
dir = os.getcwd() + r"\Cleaned"
for filename in os.listdir(dir):
    if filename.endswith(".csv") or filename.endswith(".CSV"):
        dirs.append(dir+"\\"+filename)
dirs

['c:\\Users\\David Fernandez\\Desktop\\cancerDataAnalysis\\CleansingNotebooks\\Incidence\\Cleaned\\Inc_2010.csv',
 'c:\\Users\\David Fernandez\\Desktop\\cancerDataAnalysis\\CleansingNotebooks\\Incidence\\Cleaned\\Inc_2011.csv',
 'c:\\Users\\David Fernandez\\Desktop\\cancerDataAnalysis\\CleansingNotebooks\\Incidence\\Cleaned\\Inc_2012.csv',
 'c:\\Users\\David Fernandez\\Desktop\\cancerDataAnalysis\\CleansingNotebooks\\Incidence\\Cleaned\\Inc_2013.csv',
 'c:\\Users\\David Fernandez\\Desktop\\cancerDataAnalysis\\CleansingNotebooks\\Incidence\\Cleaned\\Inc_2014.csv',
 'c:\\Users\\David Fernandez\\Desktop\\cancerDataAnalysis\\CleansingNotebooks\\Incidence\\Cleaned\\Inc_2015.csv',
 'c:\\Users\\David Fernandez\\Desktop\\cancerDataAnalysis\\CleansingNotebooks\\Incidence\\Cleaned\\Inc_2016.csv',
 'c:\\Users\\David Fernandez\\Desktop\\cancerDataAnalysis\\CleansingNotebooks\\Incidence\\Cleaned\\Inc_2017.csv',
 'c:\\Users\\David Fernandez\\Desktop\\cancerDataAnalysis\\CleansingNotebooks\\Incidence

In [4]:
for filename in dirs:
    df = pd.read_csv(filename)
    df['EGRESO'] =  pd.to_datetime(df['EGRESO'], yearfirst = True, errors = 'raise')
    df['ENTIDAD'] = df['ENTIDAD'].apply(lambda x: '{0:0>2}'.format(x))
    df['MUNIC'] = df['MUNIC'].apply(lambda x: '{0:0>3}'.format(x))
    cols = ['ENTIDAD', 'MUNIC']
    df['MUNIC'] = df[cols].apply(lambda row: ''.join(row.values.astype(str)), axis=1)
    df.columns = [x.lower() for x in df.columns]
    df['month_year_ocurr'] = df['egreso'].dt.to_period('M')
    df.rename(columns={'egreso':'date_ocurr', 'entidad':'ent_regis', 'munic':'mun_regis', 'icd':'ICD'}, inplace=True)
    df.to_csv(filename)

In [5]:
for filename in dirs:
    df = pd.read_csv(filename)
    df.rename(columns={'icd':'ICD'}, inplace=True)
    df.to_csv(filename)

In [6]:
defunc = list()
for filename in dirs:
        currDF = pd.read_csv(filename)
        defunc.append(currDF) #Append clean datafiles on list

totalDefunc = pd.concat(defunc) #Make sigle frame out of each frame in defunc list

In [8]:
pPath = dirname(dirname(os.getcwd()))+'\\codec\\TotalIncidenceTest.csv'
pPath

'c:\\Users\\David Fernandez\\Desktop\\cancerDataAnalysis\\codec\\TotalIncidenceTest.csv'

In [9]:
totalDefunc.to_csv(pPath)